In [3]:
from energieapp import db_connection, caching, mappings  # etc.
from energieapp.common_imports import *
from energieapp.frequency_utils import (
    FREQS, get_freq_minutes, get_freq_seconds, get_pandas_freq,
    check_max_rows, round_datetime_to_freq, detect_auto_frequency, resample_dataframe
)
from energieapp.db_connection import get_engine
from energieapp.mappings import group_typeid_mapping, get_typeids, validate_unique_ids
from energieapp.progress_bar_widget import ProgressBarWidget
from energieapp.caching import TTLCache

show_home_button()

engine = get_engine()
DATETIME_FORMAT = "%d/%m/%Y %H:%M"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
pio.renderers.default = 'jupyterlab'

validate_unique_ids()

progress_widget = ProgressBarWidget()

# -- CACHE-INSTANTIES --
full_data_cache = TTLCache(ttl=300)
min_max_cache   = TTLCache(ttl=300)

current_df = None
current_view = "chart"
fig_time = None

MAX_ROWS = 8000

python_aggregate = True

common_layout = widgets.Layout(width='240px', height='35px')

default_end_date = datetime.now().strftime(DATETIME_FORMAT)
default_start_date = (datetime.now() - timedelta(days=3)).strftime(DATETIME_FORMAT)

start_datetime_input = widgets.Text(
    value=default_start_date,
    placeholder='dd/mm/yyyy HH:MM',
    description='StartDatum:',
    layout=common_layout
)
end_datetime_input = widgets.Text(
    value=default_end_date,
    placeholder='dd/mm/yyyy HH:MM',
    description='EindDatum:',
    layout=common_layout
)

def parse_user_datetime(dt_str: str) -> Optional[datetime]:
    try:
        return datetime.strptime(dt_str, DATETIME_FORMAT)
    except ValueError:
        logger.error(f"Invalid date input: {dt_str}")
        return None

ean_input = widgets.Text(
    value='',
    placeholder='Vul ID/EAN in',
    description='',
    layout=widgets.Layout(width='220px', height='35px')
)

reset_filters_button = widgets.Button(
    description="Reset Filters",
    button_style="warning",
    icon="refresh",
    disabled=True,
    layout=common_layout
)
load_filters_button = widgets.Button(
    description='Zoeken',
    button_style='info',
    icon='search',
    disabled=True,
    layout=common_layout
)
generate_button = widgets.Button(
    description='Visualisatie',
    button_style='success',
    icon='line-chart',
    disabled=True,
    layout=common_layout
)

search_method_dropdown = widgets.Dropdown(
    options=[("TransferpointID", "transferpoint"),
            ("ObjectID", "objectid"),
            ("RegisterID", "registerid"),
            ("RegistratorID", "registratorid")],
    value="transferpoint",
    description="Filter:",
    layout=common_layout
)

freq_selector = widgets.Dropdown(
    options=[(f['label'], key) for key, f in FREQS.items()],
    value='auto',
    description='Frequentie:',
    layout=common_layout
)
chart_type_selector = widgets.Dropdown(
    options=[('Lijn', 'line'),
            ('Staaf', 'bar')],
    value='line',
    description='Grafiek:',
    layout=common_layout
)

warning_message = widgets.HTML("")
quick_fix_freq_button = widgets.Button(
    description="Wijzig freq -> 1 uur",
    button_style="warning",
    icon="clock-o",
    layout=widgets.Layout(width='160px', height='35px')
)
quick_fix_date_button = widgets.Button(
    description="Beperk datumbereik",
    button_style="warning",
    icon="calendar",
    layout=widgets.Layout(width='160px', height='35px')
)
warning_container = widgets.VBox([], layout=widgets.Layout(margin="5px 0px"))

def validate_data_request(change=None):
    start_dt_str = start_datetime_input.value
    end_dt_str = end_datetime_input.value
    start_dt = parse_user_datetime(start_dt_str)
    end_dt = parse_user_datetime(end_dt_str)
    now = datetime.now()

    error_messages = []

    if not start_dt:
        error_messages.append("Ongeldige startdatum/tijd (dd/mm/yyyy HH:MM)!")
    elif start_dt > now:
        error_messages.append("Startdatum mag niet in de toekomst liggen!")

    if not end_dt:
        error_messages.append("Ongeldige einddatum/tijd (dd/mm/yyyy HH:MM)!")
    elif end_dt > now:
        error_messages.append("Einddatum mag niet in de toekomst liggen!")

    if start_dt and end_dt and end_dt < start_dt:
        error_messages.append("Einddatum mag niet vóór de startdatum liggen!")

    if error_messages:
        warning_message.value = "<span style='color:red;font-weight:bold;'>" + "<br>".join(error_messages) + "</span>"
        warning_container.children = [warning_message]
        load_filters_button.disabled = True
        generate_button.disabled = True
        return
    else:
        warning_message.value = ""
        warning_container.children = []
        load_filters_button.disabled = (ean_input.value.strip() == "")
        generate_button.disabled = (ean_input.value.strip() == "")

    freq_key = freq_selector.value
    if freq_key != 'auto' and start_dt and end_dt:
        valid, expected_rows = check_max_rows(start_dt, end_dt, freq_key, MAX_ROWS)
        if not valid:
            warning_message.value = (
                f"<span style='color:red;font-weight:bold;'>"
                f"Teveel data (~{int(expected_rows)} rijen). "
                "Verklein bereik of kies hogere resolutie!</span>"
            )
            warning_container.children = [
                warning_message,
                widgets.HBox([quick_fix_freq_button, quick_fix_date_button],
                            layout=widgets.Layout(justify_content='center'))
            ]
            load_filters_button.disabled = True
            generate_button.disabled = True
            return

start_datetime_input.observe(validate_data_request, names="value")
end_datetime_input.observe(validate_data_request, names="value")
freq_selector.observe(validate_data_request, names="value")

def quick_fix_freq_action(b):
    freq_selector.value = 'H'
    validate_data_request()

def quick_fix_date_action(b):
    start_dt = parse_user_datetime(start_datetime_input.value)
    if not start_dt:
        return
    freq_key = freq_selector.value
    max_duration = (MAX_ROWS - 1) * get_freq_seconds(freq_key or 'H')
    new_end_dt = start_dt + timedelta(seconds=max_duration)
    end_datetime_input.value = new_end_dt.strftime(DATETIME_FORMAT)
    validate_data_request()

quick_fix_freq_button.on_click(quick_fix_freq_action)
quick_fix_date_button.on_click(quick_fix_date_action)
group_checkbox_container = widgets.VBox([])
group_accordion = widgets.Accordion(children=[group_checkbox_container])
group_accordion.set_title(0, "Selecteer kanalen")
include_status_checkbox = widgets.Checkbox(
    value=True,
    description='Inclusief Status per Register',
    layout=widgets.Layout(margin="2px 0px 2px 0px")
)
aggregate_selector = widgets.Checkbox(
    value=False,
    description='Toon totalen (geaggregeerd)',
    layout=widgets.Layout(margin="2px 0px 2px 0px")
)
interval_value_checkbox = widgets.Checkbox(
    value=False,
    description='Toon waarde per interval',
    layout=widgets.Layout(margin="2px 0px 2px 0px")
)

def on_agg_toggle_change(change):
    pass

aggregate_selector.observe(on_agg_toggle_change, names='value')

compare_toggle = widgets.Checkbox(
    value=False,
    description='Vergelijking inschakelen',
    layout=widgets.Layout(margin="2px 0px 2px 0px")
)
compare_group1_dropdown = widgets.Dropdown(options=[], description="Groep 1:", layout=common_layout)
compare_group2_dropdown = widgets.Dropdown(options=[], description="Groep 2:", layout=common_layout)
compare_options_container = widgets.HBox([compare_group1_dropdown, compare_group2_dropdown],
                                        layout=widgets.Layout(justify_content='center'))
compare_options_container.layout.display = 'none'

def on_compare_toggle_change(change):
    if change['new']:
        compare_options_container.layout.display = 'flex'
    else:
        compare_options_container.layout.display = 'none'

compare_toggle.observe(on_compare_toggle_change, names='value')

options_container = widgets.VBox([
    aggregate_selector,
    interval_value_checkbox,
    widgets.HTML("<b>Vergelijking</b>"),
    compare_toggle,
    compare_options_container
])
options_accordion = widgets.Accordion(children=[options_container])
options_accordion.set_title(0, "Opties")

output = widgets.Output()
fig_container = widgets.VBox(layout=widgets.Layout(width='100%'))

def on_ean_input_change(change):
    value = change['new'].strip()
    load_filters_button.disabled = (value == "") or bool(warning_container.children)

ean_input.observe(on_ean_input_change, names='value')

def reset_filters(b):
    logger.info("Filters resetten...")
    ean_input.value = ''
    search_method_dropdown.value = 'transferpoint'
    start_datetime_input.value = (datetime.now() - timedelta(days=3)).strftime(DATETIME_FORMAT)
    end_datetime_input.value = datetime.now().strftime(DATETIME_FORMAT)
    freq_selector.value = 'auto'
    chart_type_selector.value = 'line'
    for cb in group_checkbox_container.children:
        if isinstance(cb, widgets.Checkbox):
            cb.value = True
    aggregate_selector.value = False
    interval_value_checkbox.value = False
    compare_group1_dropdown.value = None
    compare_group2_dropdown.value = None
    compare_toggle.value = False

    generate_button.disabled = True
    reset_filters_button.disabled = True
    load_filters_button.disabled = True
    warning_message.value = ""
    warning_container.children = []

    with output:
        clear_output()
        print("Filters zijn gereset naar de standaardwaarden.")
reset_filters_button.on_click(reset_filters)
def update_compare_dropdown_options(*args):
    selected_groups = [cb.description for cb in group_checkbox_container.children if cb.value]
    compare_group1_dropdown.options = selected_groups
    compare_group2_dropdown.options = selected_groups

def load_filters(ean_val: str):
    with output:
        clear_output()
        print("Filters laden...")

    available_typeids = fetch_typeids_for_ean(ean_val)
    if not available_typeids:
        with output:
            clear_output()
            print(f"Geen TypeIds gevonden voor waarde {ean_val}.")
        group_checkbox_container.children = []
        return

    relevant_groups = [
        grp for grp, tid_list in group_typeid_mapping.items()
        if set(tid_list) & available_typeids
    ]
    if not relevant_groups:
        with output:
            clear_output()
            print(f"Geen relevante groepen gevonden voor waarde {ean_val}.")
        group_checkbox_container.children = []
        return

    sorted_groups = sorted(relevant_groups)
    group_checkboxes = []
    for grp in sorted_groups:
        cb = widgets.Checkbox(value=True, description=grp, indent=False)
        cb.observe(update_compare_dropdown_options, 'value')
        group_checkboxes.append(cb)
    group_checkbox_container.children = group_checkboxes
    group_accordion.selected_index = 0
    compare_group1_dropdown.options = sorted_groups
    compare_group2_dropdown.options = sorted_groups
    update_compare_dropdown_options()
    generate_button.disabled = False
    reset_filters_button.disabled = False

    with output:
        clear_output()
        print(f"Filters geladen voor waarde {ean_val}. Selecteer kanalen: {sorted_groups}")

def on_load_filters_clicked(button):
    ean_val = ean_input.value.strip()
    if not ean_val:
        with output:
            clear_output()
            print("Vul eerst een waarde in.")
        return
    logger.info("Filters laden voor waarde: %s", ean_val)
    load_filters(ean_val)

load_filters_button.on_click(on_load_filters_clicked)

def fetch_typeids_for_ean(ean_value: str) -> set:
    logger.info("Ophalen van TypeIDs voor waarde...")
    method = search_method_dropdown.value
    if method == "transferpoint":
        query = """
            SELECT DISTINCT r.TypeId
            FROM TBL_Register r
            JOIN TBL_ConnectionPoint cp ON cp.ID = r.ConnectionPointId
            WHERE cp.EAN_ConnectionPoint = ?
                OR cp.TransferPointID IN (
                    SELECT ID FROM TBL_ConnectionPoint
                    WHERE EAN_ConnectionPoint = ?
                )
        """
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(query, conn, params=(ean_value, ean_value))
    elif method == "objectid":
        query = """
            SELECT DISTINCT r.TypeId
            FROM TBL_Register r
            JOIN TBL_ConnectionPoint cp ON cp.ID = r.ConnectionPointId
            WHERE cp.ObjectId = (
                SELECT TOP 1 cp2.ObjectId
                FROM TBL_ConnectionPoint cp2
                WHERE cp2.EAN_ConnectionPoint = ?
            )
        """
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(query, conn, params=(ean_value,))
    elif method == "registerid":
        query = """
            SELECT DISTINCT TypeId
            FROM TBL_Register
            WHERE ID = ?
        """
        register_id = int(ean_value)
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(query, conn, params=(register_id,))
    elif method == "registratorid":
        query = """
            SELECT DISTINCT r.TypeId
            FROM TBL_Register r
            WHERE r.RegistratorID = ?
        """
        registrator_id = int(ean_value)
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(query, conn, params=(registrator_id,))
    else:
        return set()

    if df_temp.empty:
        return set()
    return set(df_temp['TypeId'].unique())

def fetch_min_max_period(ean_value: str, allowed_typeids_str: str,
                        start_date_str: str, end_date_str: str) -> tuple:
    logger.info("Ophalen van min/max periode (UTC)...")
    search_method = search_method_dropdown.value
    cache_key = (ean_value, allowed_typeids_str, start_date_str, end_date_str, search_method, 'minmax')
    cached = min_max_cache.get(cache_key)
    if cached:
        logger.info("Min/Max periode uit cache gehaald.")
        return cached

    sp_query = """
        EXEC [dbo].[usp_GetMinMaxPeriodForEAN]
            @EAN_ConnectionPoint = ?,
            @AllowedTypeIDs = ?,
            @StartDateStr = ?,
            @EndDateStr = ?,
            @SearchMethod = ?
    """
    with engine.connect() as conn:
        df_temp = pd.read_sql_query(sp_query, conn,
                                    params=(ean_value, allowed_typeids_str,
                                            start_date_str, end_date_str, search_method))
    if df_temp.empty or pd.isnull(df_temp['MinUTCPeriod'].iloc[0]):
        result = (None, None)
    else:
        result = (df_temp['MinUTCPeriod'].iloc[0], df_temp['MaxUTCPeriod'].iloc[0])
    min_max_cache.set(cache_key, result)
    return result

def fetch_full_data(ean_value: str, allowed_typeids_str: str,
                    start_date_str: str, end_date_str: str,
                    interval_minutes: int,
                    include_status: bool = False) -> Optional[pd.DataFrame]:
    logger.info("Ophalen data uit usp_GetConnectionDataFull; Interval=%dmin, status=%s",
                interval_minutes, include_status)
    search_method = search_method_dropdown.value
    cache_key = (ean_value, allowed_typeids_str, start_date_str, end_date_str,
                search_method, 'pivot', interval_minutes, include_status)
    cached = full_data_cache.get(cache_key)
    if cached is not None:
        logger.info("Volledige data uit cache gehaald.")
        return cached

    sp_query = """
        EXEC [dbo].[usp_GetConnectionDataFull]
            @EAN_ConnectionPoint = ?,
            @AllowedTypeIDs = ?,
            @StartDateStr = ?,
            @EndDateStr = ?,
            @SearchMethod = ?,
            @IntervalMinutes = ?,
            @IncludeStatus = ?
    """
    with engine.connect() as conn:
        df = pd.read_sql_query(
            sp_query, conn,
            params=(ean_value, allowed_typeids_str, start_date_str,
                    end_date_str, search_method, interval_minutes, int(include_status)),
            parse_dates=['utcperiod']
        )

    result = df if not df.empty else None
    full_data_cache.set(cache_key, result)
    return result

def group_columns_by_typeid_agg(df: pd.DataFrame, engine,
                                group_typeid_mapping: dict,
                                selected_groups: list,
                                include_status: bool) -> pd.DataFrame:
    df_agg = df.copy()
    if 'utcperiod' not in df_agg.columns:
        raise ValueError("DataFrame moet een 'utcperiod' kolom bevatten.")
    df_agg['utcperiod'] = pd.to_datetime(df_agg['utcperiod'])
    df_agg.set_index('utcperiod', inplace=True)

    register_ids = set()
    for col in df_agg.columns:
        match = re.search(r'\((\d+)\)', col)
        if match:
            register_ids.add(int(match.group(1)))
    if register_ids:
        query = f"SELECT ID, TypeId FROM dbo.TBL_Register WHERE ID IN ({','.join(map(str, register_ids))})"
        with engine.connect() as conn:
            mapping_df = pd.read_sql_query(query, conn)
        registerid_to_typeid = dict(zip(mapping_df['ID'], mapping_df['TypeId']))
    else:
        registerid_to_typeid = {}

    agg_data = {}
    index = df_agg.index
    for group_name in selected_groups:
        typeid_list = group_typeid_mapping.get(group_name, [])
        cons_cols = []
        status_cols = []
        for col in df_agg.columns:
            match = re.search(r'\((\d+)\)', col)
            if not match:
                continue
            reg_id = int(match.group(1))
            if registerid_to_typeid.get(reg_id) in typeid_list:
                if "(status)" in col.lower():
                    status_cols.append(col)
                else:
                    cons_cols.append(col)
        if cons_cols:
            agg_data[group_name + " Total"] = df_agg[cons_cols].sum(axis=1)
        else:
            agg_data[group_name + " Total"] = pd.Series(0, index=index)
        if include_status and status_cols:
            def row_status(row):
                statuses = row[status_cols].tolist()
                if any(s == 'P' for s in statuses):
                    return 'P'
                elif any(s == 'T' for s in statuses):
                    return 'T'
                else:
                    return ''
            agg_data[group_name + " Status"] = df_agg.apply(row_status, axis=1)
        else:
            agg_data[group_name + " Status"] = pd.Series([''] * len(df_agg), index=index)

    df_result = pd.DataFrame(agg_data, index=index)
    return df_result

def fetch_and_prepare_data(ean_val: str,
                            chosen_groups: list,
                            start_dt: datetime,
                            end_dt: datetime,
                            freq_val: str,
                            agg: bool) -> Optional[pd.DataFrame]:
    logger.info("Data voorbereiden: ean=%s, freq=%s, agg=%s, groups=%s",
                ean_val, freq_val, agg, chosen_groups)

    typeids_final = []
    for grp in chosen_groups:
        typeids_final.extend(group_typeid_mapping.get(grp, []))
    if not typeids_final:
        with output:
            clear_output()
            print("Geen TypeIds gevonden uit de gekozen groepen!")
        return None
    allowed_typeids_str = ",".join(str(t) for t in set(typeids_final))

    interval_minutes = get_freq_minutes(freq_val)
    start_date_str = start_dt.strftime(DATETIME_FORMAT)
    end_date_str  = end_dt.strftime(DATETIME_FORMAT)

    min_period, max_period = fetch_min_max_period(ean_val, allowed_typeids_str, start_date_str, end_date_str)
    if not min_period or not max_period:
        with output:
            clear_output()
            print("Geen data in deze periode (min/max is NULL).")
        return None

    want_status = include_status_checkbox.value
    df_full = fetch_full_data(ean_val, allowed_typeids_str,
                                start_date_str, end_date_str,
                                interval_minutes,
                                include_status=want_status)
    if df_full is None or df_full.empty:
        with output:
            clear_output()
            print("Geen data of fout bij ophalen van data.")
        return None

    numeric_cols = []
    for c in df_full.columns:
        if c.lower() != 'utcperiod' and '(status)' not in c.lower():
            numeric_cols.append(c)
    df_full[numeric_cols] = df_full[numeric_cols].fillna(0)

    df_filtered = df_full[(df_full['utcperiod'] >= start_dt) & (df_full['utcperiod'] <= end_dt)].copy()
    df_filtered.set_index('utcperiod', inplace=True)

    if not python_aggregate:
        df_resampled = df_filtered
    else:
        if freq_val == 'auto':
            df_resampled = df_filtered
        else:
            pandas_freq = get_pandas_freq(freq_val)
            if not pandas_freq:
                df_resampled = df_filtered
            else:
                if agg:
                    df_reset = df_filtered.reset_index()
                    df_grouped = group_columns_by_typeid_agg(df_reset, engine,
                                                            group_typeid_mapping,
                                                            chosen_groups,
                                                            include_status=want_status)
                    def agg_status(series):
                        return 'P' if (series=='P').any() else ('T' if (series=='T').any() else '')
                    agg_dict = {}
                    for col in df_grouped.columns:
                        if col.endswith("Total"):
                            agg_dict[col] = 'sum'
                        elif col.endswith("Status"):
                            agg_dict[col] = agg_status
                    df_resampled = df_grouped.resample(pandas_freq).agg(agg_dict)
                else:
                    df_resampled = df_filtered.resample(pandas_freq).sum()

    df_resampled.index.name = "UTC period"
    logger.info("Data succesvol voorbereid (%d rijen).", len(df_resampled))
    return df_resampled

def on_generate_visual_clicked(b):
    logger.info("Visualisatie genereren gestart...")
    generate_time_series()

generate_button.on_click(on_generate_visual_clicked)

def generate_time_series():
    global current_df, fig_time, current_view
    progress_widget.show(status="Visualisatie genereren...")
    generate_button.disabled = True
    load_filters_button.disabled = True

    with output:
        clear_output()
        print("Visualisatie genereren...")

    ean_val = ean_input.value.strip()
    if not ean_val:
        with output:
            clear_output()
            print("Vul EAN in")
        progress_widget.update(100, "Fout: Geen waarde", error=True)
        progress_widget.finish()
        return

    chosen_groups = [cb.description for cb in group_checkbox_container.children if cb.value]
    if not chosen_groups:
        with output:
            clear_output()
            print("Geen groepen geselecteerd.")
        progress_widget.update(100, "Fout: Geen groepen geselecteerd", error=True)
        progress_widget.finish()
        return

    start_dt = parse_user_datetime(start_datetime_input.value)
    end_dt  = parse_user_datetime(end_datetime_input.value)
    if not start_dt or not end_dt:
        with output:
            clear_output()
            print("Ongeldige datum/tijd (dd/mm/yyyy HH:MM).")
        progress_widget.update(100, "Fout: Ongeldige datum/tijd", error=True)
        progress_widget.finish()
        return

    freq_val = freq_selector.value
    agg_val  = aggregate_selector.value
    chart_type = chart_type_selector.value

    progress_widget.update(30, "Data ophalen...")
    df_resampled = fetch_and_prepare_data(ean_val, chosen_groups, start_dt, end_dt, freq_val, agg_val)
    if df_resampled is None or df_resampled.empty:
        progress_widget.update(100, "Geen data gevonden", error=True)
        progress_widget.finish()
        return

    df_resampled = df_resampled[df_resampled.sum(numeric_only=True).sort_values(ascending=False).index]
    current_df = df_resampled
    current_view = "chart"

    if chart_type == 'bar':
        fig_time = go.FigureWidget(
            layout=go.Layout(
                autosize=True,
                title=dict(
                    text=f"Energiemonitor {ean_val}",
                    y=0.95,
                    x=0.5,
                    xanchor='center',
                    yanchor='top',
                    font=dict(size=22, color='darkblue')
                ),
                xaxis=dict(
                    title="Tijd",
                    tickangle=-45,
                    showgrid=True,
                    gridcolor='rgba(200,200,200,0.3)',
                    gridwidth=1
                ),
                yaxis=dict(
                    title="Energie (kWh)",
                    showgrid=True,
                    gridcolor='rgba(200,200,200,0.3)',
                    gridwidth=1
                ),
                template="plotly_white",
                hovermode="x unified",
                hoverlabel=dict(bgcolor='rgba(0,0,0,0.8)', font=dict(color='white')),
                legend=dict(
                    orientation="h",
                    yanchor="top",
                    y=-0.2,
                    xanchor="center",
                    x=0.5,
                    bgcolor='rgba(255,255,255,0.7)',
                    bordercolor='Black',
                    borderwidth=1
                ),
                paper_bgcolor='rgba(255,255,255,1)',
                plot_bgcolor='rgba(245,245,245,1)',
                height=700,
                margin=dict(l=60, r=40, t=100, b=150),
                dragmode='zoom'
            )
        )

        color_map_special = {
            "Hoofdmeting elektriciteit ODN Total": "blue",
            "Bruto productie Total": "red"
        }
        default_colors = [
            "orange", "green", "purple", "teal",
            "cyan", "magenta", "brown", "gold",
            "darkred", "navy"
        ]
        chosen_cols = df_resampled.columns.tolist()
        col_color_map = {}
        c_idx = 0
        for col_name in chosen_cols:
            if col_name in color_map_special:
                col_color_map[col_name] = color_map_special[col_name]
            else:
                if c_idx >= len(default_colors):
                    c_idx = 0
                col_color_map[col_name] = default_colors[c_idx]
                c_idx += 1

        all_traces = []
        for col_name in chosen_cols:
            line_color = col_color_map[col_name]
            col_total = df_resampled[col_name].sum() if df_resampled[col_name].dtype in [np.float64, np.float32, np.int64, np.int32] else 0
            trace_legend_name = f"{col_name} (Totaal: {col_total:.2f})" if col_total else col_name

            bar_trace = go.Bar(
                x=df_resampled.index,
                y=df_resampled[col_name],
                name=trace_legend_name,
                marker=dict(color=line_color),
                text=(df_resampled[col_name].round(2).astype(str) if interval_value_checkbox.value else None),
                textposition='outside' if interval_value_checkbox.value else None,
                hovertemplate='%{y:.2f} kWh<extra></extra>'
            )
            all_traces.append(bar_trace)

        if (compare_toggle.value
                and compare_group1_dropdown.value
                and compare_group2_dropdown.value
                and compare_group1_dropdown.value != compare_group2_dropdown.value):
            pass

        for t in all_traces:
            fig_time.add_trace(t)

    else:
        fig_time = go.FigureWidget(
            layout=go.Layout(
                autosize=True,
                title=dict(
                    text=f"Energiemonitor {ean_val}",
                    y=0.95,
                    x=0.5,
                    xanchor='center',
                    yanchor='top',
                    font=dict(size=22, color='darkblue')
                ),
                xaxis=dict(
                    title="Tijd",
                    tickangle=-45,
                    showgrid=True,
                    gridcolor='rgba(200,200,200,0.3)',
                    gridwidth=1
                ),
                yaxis=dict(
                    title="Energie (kWh)",
                    showgrid=True,
                    gridcolor='rgba(200,200,200,0.3)',
                    gridwidth=1
                ),
                template="plotly_white",
                hovermode="x unified",
                hoverlabel=dict(bgcolor='rgba(0,0,0,0.8)', font=dict(color='white')),
                legend=dict(
                    orientation="h",
                    yanchor="top",
                    y=-0.2,
                    xanchor="center",
                    x=0.5,
                    bgcolor='rgba(255,255,255,0.7)',
                    bordercolor='Black',
                    borderwidth=1
                ),
                paper_bgcolor='rgba(255,255,255,1)',
                plot_bgcolor='rgba(245,245,245,1)',
                height=700,
                margin=dict(l=60, r=40, t=100, b=150),
                dragmode='zoom'
            )
        )

        color_map_special = {
            "Hoofdmeting elektriciteit ODN Total": "blue",
            "Bruto productie Total": "red"
        }
        default_colors = [
            "orange", "green", "purple", "teal",
            "cyan", "magenta", "brown", "gold",
            "darkred", "navy"
        ]
        chosen_cols = current_df.columns.tolist()
        col_color_map = {}
        c_idx = 0
        for col_name in chosen_cols:
            if col_name in color_map_special:
                col_color_map[col_name] = color_map_special[col_name]
            else:
                if c_idx >= len(default_colors):
                    c_idx = 0
                col_color_map[col_name] = default_colors[c_idx]
                c_idx += 1

        all_traces = []
        for col_name in chosen_cols:
            line_color = col_color_map[col_name]
            col_total = current_df[col_name].sum() if current_df[col_name].dtype in [np.float64, np.float32, np.int64, np.int32] else 0
            trace_legend_name = f"{col_name} (Totaal: {col_total:.2f})" if col_total else col_name

            if include_status_checkbox.value:
                if agg_val:
                    status_col = col_name.replace("Total", "Status")
                else:
                    status_col = col_name.replace("(consumption)", "(status)")
            else:
                status_col = None

            symbol_array = []
            size_array = []

            if status_col and status_col in current_df.columns:
                for status_val in current_df[status_col]:
                    if status_val == 'T':
                        symbol_array.append('x')
                        size_array.append(10)
                    elif status_val == 'P':
                        symbol_array.append('triangle-up')
                        size_array.append(10)
                    else:
                        symbol_array.append('circle')
                        size_array.append(2)
            else:
                symbol_array = ["circle"] * len(current_df)
                size_array  = [2] * len(current_df)

            trace = go.Scatter(
                x=current_df.index,
                y=current_df[col_name],
                mode='lines+markers' + ('+text' if interval_value_checkbox.value else ''),
                line=dict(color=line_color, width=2),
                name=trace_legend_name,
                text=(current_df[col_name].round(2).astype(str) if interval_value_checkbox.value else None),
                textposition='top center' if interval_value_checkbox.value else None,
                hovertemplate='%{y:.2f} kWh' if interval_value_checkbox.value else '%{y}',
                marker=dict(
                    symbol=symbol_array,
                    size=size_array,
                    color='white',
                    line=dict(width=1, color='black')
                )
            )
            all_traces.append(trace)

            if (compare_toggle.value
                    and compare_group1_dropdown.value
                    and compare_group2_dropdown.value
                    and compare_group1_dropdown.value != compare_group2_dropdown.value):

                pos_group = compare_group1_dropdown.value
                neg_group = compare_group2_dropdown.value
                pos_col = f"{pos_group} Total"
                neg_col = f"{neg_group} Total"

                if pos_col in current_df.columns and neg_col in current_df.columns:
                    trace_pos, trace_neg = None, None
                    for t in all_traces:
                        if pos_col in t.name:
                            trace_pos = t
                        if neg_col in t.name:
                            trace_neg = t

                    if trace_pos and trace_neg:
                        trace_neg.hoverinfo = 'skip'
                        trace_neg.hovertemplate = None

                        series_pos = current_df[pos_col]
                        series_neg = current_df[neg_col]

                        diff = series_pos - series_neg
                        percdiff = np.where(series_pos != 0, diff / series_pos * 100, 0)

                        cdata = np.column_stack((series_neg, diff, percdiff))
                        trace_pos.customdata = cdata
                        trace_pos.hovertemplate = (
                            f"{pos_group}: %{{y:.2f}} kWh<br>"
                            f"{neg_group}: %{{customdata[0]:.2f}} kWh<br>"
                            "Verschil: %{customdata[1]:.2f} kWh<br>"
                            "Percentueel: %{customdata[2]:.2f}%<extra></extra>"
                        )

                        diff_pos = diff.clip(lower=0)
                        fill_trace_pos = go.Scatter(
                            x=current_df.index.tolist() + current_df.index[::-1].tolist(),
                            y=list(series_pos) + list((series_pos - diff_pos)[::-1]),
                            fill='toself',
                            fillcolor="rgba(0,255,0,0.2)",
                            line=dict(color='rgba(0,0,0,0)'),
                            name=f"{pos_group} > {neg_group}",
                            showlegend=True,
                            hoverinfo='skip',
                            opacity=0.3
                        )
                        all_traces.append(fill_trace_pos)

                        diff_neg = diff.clip(upper=0)
                        fill_trace_neg = go.Scatter(
                            x=current_df.index.tolist() + current_df.index[::-1].tolist(),
                            y=list(series_pos - diff_neg) + list(series_pos[::-1]),
                            fill='toself',
                            fillcolor="rgba(255,0,0,0.2)",
                            line=dict(color='rgba(0,0,0,0)'),
                            name=f"{neg_group} > {pos_group}",
                            showlegend=True,
                            hoverinfo='skip',
                            opacity=0.3
                        )
                        all_traces.append(fill_trace_neg)

        for t in all_traces:
            fig_time.add_trace(t)
    fig_container.children = [fig_time]

    progress_widget.update(100, "Klaar!")
    with output:
        clear_output()
        print(f"Visualisatie succesvol gegenereerd ({len(current_df)} rijen).")

    generate_button.disabled = False
    load_filters_button.disabled = False
    progress_widget.finish()

def toggle_filters_display(b):
    if filters_container.layout.display == 'none':
        filters_container.layout.display = 'block'
        toggle_filters_button.description = "Verberg"
        toggle_filters_button.icon = "chevron-up"
    else:
        filters_container.layout.display = 'none'
        toggle_filters_button.description = "Toon filters"
        toggle_filters_button.icon = "chevron-down"
    if fig_time is not None:
        fig_time.update_layout(autosize=True)

toggle_filters_button = widgets.Button(
    description="Verberg",
    icon='chevron-up',
    button_style='info',
    layout=widgets.Layout(width='120px', height='35px')
)
toggle_filters_button.on_click(toggle_filters_display)

filters_container = widgets.VBox([
    widgets.HBox(
        [search_method_dropdown, ean_input, load_filters_button, reset_filters_button,
         generate_button],
        layout=widgets.Layout(gap="5px", align_items='center', flex_flow="row wrap")
    ),
    widgets.HBox(
        [start_datetime_input, end_datetime_input, freq_selector, chart_type_selector],
        layout=widgets.Layout(gap="5px", align_items='center', flex_flow="row wrap")
    ),
    warning_container,
    widgets.HBox([group_accordion, options_accordion],
                 layout=widgets.Layout(justify_content='center', gap="10px")),
    progress_widget.widget(),
    output
], layout=widgets.Layout(width='100%', padding="10px"))

top_section = widgets.VBox([toggle_filters_button, filters_container],
                            layout=widgets.Layout(width='100%', padding="0px"))
view_container = widgets.VBox([fig_container],
                             layout=widgets.Layout(width='100%'))
final_ui = widgets.VBox([top_section, view_container],
                         layout=widgets.Layout(width='100%', height='auto', padding="10px"))

def show_chart():
    fig_container.layout.display = 'block'

show_chart()
display(final_ui)

ModuleNotFoundError: No module named 'energieapp'